In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nengo
from nengo.utils.matplotlib import rasterplot
import phd

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

def img(array):
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.colorbar()
    sns.despine()
    plt.tight_layout()

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

In [ ]:
model = phd.sermo.AudioFeatures()
model.audio = 'sprachsynthese.wav'
model.freqs = phd.filters.erbspace(20, 4000, 64)
model.n_cepstra = 13

In [ ]:
# Get MFCCs from scikit-speech
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape

In [ ]:
img(x)
plt.title("Mel-frequency cepstral coefficients")

In [ ]:
from scipy.stats import zscore
img(zscore(x, axis=0))

In [ ]:
# Get NCCs from Sermo
model.periphery.auditory_filter = phd.filters.gammatone(model.freqs)
net = model.build()

with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.cepstra.output, synapse=0.01)

sim = nengo.Simulator(net, dt=0.001)
sim.run(1.4)

In [ ]:
phd.plots.cochleogram(sim.data[ihc_p], sim.trange(), model.freqs)

In [ ]:
phd.plots.cochleogram(sim.data[an_in_p], sim.trange(), model.freqs)

In [ ]:
rasterplot(sim.trange(), sim.data[an_p])
plt.ylim(0, net.periphery.an.n_neurons * net.periphery.an.n_ensembles);

In [ ]:
img(sim.data[c_p])

In [ ]:
img(zscore(sim.data[c_p], axis=0))

## With derivatives

## Phoneme classification with SVM